In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Notebook is in progress

# Stock Prediction

# Read Data

In [ ]:
df=pd.read_csv('../input/nyse/prices.csv')

# Describe the data

In [ ]:
df.head()

# Check for Null Values

In [ ]:
df.isnull().sum()

***No Null values in the entire dataset***

# Check Datatype of the features

In [ ]:
df.info()

***Date & Symbol are in object datatype and rest are float datatype***

In [ ]:
df.describe()

# Dataframe shape

In [ ]:
df.shape

In [ ]:
df['symbol'].nunique()

***There are 501 symbols in the datasets and in total there are 851264 rows with 7 columns***  
***Data set is good for analysis. however, since there are many company details are there. we will sample it with one Company***

# Sampling AMAZON from the dataset
GOOGL is the NYSE stock symbol for Google stocks. let us take that as a sample for our analysis.

In [ ]:
df1=df[df['symbol']=='AMZN']

In [ ]:
df1.head()

In [ ]:
df1.describe()

In [ ]:
np.round(df1.median(),2)

***All the Median of all the numeric values are lesser than mean, so the dataset is right skewed

In [ ]:
df1['date']=pd.to_datetime(df1['date'])

In [ ]:
print("Minimum date value : {}".format(df1['date'].min()))
print("Maximum date value : {}".format(df1['date'].max()))

***we have around 6 years of data***

# EDA

In [ ]:
#importing ploting libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
colors = ['#FF9900','#000000']
sns.set(palette=colors, font='Serif', style='white', rc={'axes.facecolor':'whitesmoke', 'figure.facecolor':'whitesmoke'})
sns.palplot(colors, size=2)

In [ ]:
fig=plt.figure(figsize=(20,8))
ax=sns.lineplot(data=df1, x='date',y='open')
ax=sns.lineplot(data=df1, x='date',y='close', color=colors[1]);
for s in ['left','right','top','bottom']:
    ax.spines[s].set_visible(False)

plt.title("AMAZON Stock value changes since 2010", size=20, weight='bold')

In [ ]:
fig=plt.figure(figsize=(20,8))
ax=sns.lineplot(data=df1, x='date',y='volume')
#ax=sns.lineplot(data=df1, x='date',y='close', color=colors[1]);
for s in ['left','right','top','bottom']:
    ax.spines[s].set_visible(False)
plt.title("Google Stock volume", size=20, weight='bold')

# Univariated Analysis

In [ ]:
#integer columns
fig=plt.figure(figsize=(20,8), tight_layout=True)
plt.suptitle("Analysing the Numeric variables", size=20, weight='bold')
ax=fig.subplot_mosaic("""AB
                         CC
                         DE""")
sns.kdeplot(df1['high'], ax=ax['A'], color=colors[0], fill=True, linewidth=2)
sns.kdeplot(df1['low'], ax=ax['B'], color=colors[1],fill=True, linewidth=2)
sns.kdeplot(df1['open'], ax=ax['C'], color=colors[0],fill=True, linewidth=2)
sns.kdeplot(df1['close'], ax=ax['D'], color=colors[1],fill=True, linewidth=2)
sns.kdeplot(df1['volume'], ax=ax['E'], color=colors[0],fill=True, linewidth=2)
ax['B'].yaxis.set_visible(False)
ax['E'].yaxis.set_visible(False)
ax['A'].yaxis.label.set_alpha(0.5)
ax['C'].yaxis.label.set_alpha(0.5)
ax['A'].yaxis.label.set_alpha(0.5)
ax['C'].yaxis.label.set_alpha(0.5)
ax['D'].yaxis.label.set_alpha(0.5)
for s in ['left','right','top','bottom']:
    ax['A'].spines[s].set_visible(False)
    ax['B'].spines[s].set_visible(False)
    ax['C'].spines[s].set_visible(False)
    ax['D'].spines[s].set_visible(False)
    ax['E'].spines[s].set_visible(False)

In [ ]:
#integer columns
fig=plt.figure(figsize=(20,8), tight_layout=True)
plt.suptitle("Boxplot the Numeric variables", size=20, weight='bold')
ax=fig.subplot_mosaic("""AB
                         CC
                         DE""")
sns.boxplot(df1['high'], ax=ax['A'], color=colors[0])
sns.boxplot(df1['low'], ax=ax['B'], color=colors[1])
sns.boxplot(df1['open'], ax=ax['C'], color=colors[0])
sns.boxplot(df1['close'], ax=ax['D'], color=colors[1])
sns.boxplot(df1['volume'], ax=ax['E'], color=colors[1])
ax['B'].yaxis.set_visible(False)
ax['E'].yaxis.set_visible(False)
ax['A'].yaxis.label.set_alpha(0.5)
ax['C'].yaxis.label.set_alpha(0.5)
ax['A'].yaxis.label.set_alpha(0.5)
ax['C'].yaxis.label.set_alpha(0.5)
ax['D'].yaxis.label.set_alpha(0.5)
for s in ['left','right','top','bottom']:
    ax['A'].spines[s].set_visible(False)
    ax['B'].spines[s].set_visible(False)
    ax['C'].spines[s].set_visible(False)
    ax['D'].spines[s].set_visible(False)
    ax['E'].spines[s].set_visible(False)

***There seems to be ouliers in the data set, however, those values can't be considered as outlier as they may be extrem values during peak selling days***  
***Date column has been ingored as it is series of numbers***  
Let us analyse more for conclusion

# Categorical feature analysis  
** There is no categorical feature other than Symbol. as we have taken the Google stock as sample data, we will drop the Symbol feature.

In [ ]:
df1.drop(['symbol'], axis=1, inplace=True)

# Bivariated & Multivariated Analysis

In [ ]:
# we need to predict the closing price of the stock, lets us consider 'Close' feature as the Target variable. 
sns.pairplot(df1,corner=True)

In [ ]:
df1.corr()['close']

# Hypothesis test to find the Normality in the Dataset

In [ ]:
from scipy.stats import levene, shapiro
int_cols=df1.select_dtypes(exclude='object').columns.to_list()

for i in int_cols:
    _, p_value=shapiro(df1[i])
    if p_value<0.05:
        print("Feature {} is normaly distributed".format(i))
    else:
        print("Feature {} is not normaly distributed".format(i))
        
    print("Normalitiy test p_value for featue -  {} is {}".format(i,np.round(p_value,3)))

# Correlation

In [ ]:
fig=plt.figure(figsize=(15,8))
sns.heatmap(df1.corr(), annot=True, cmap=[colors[0],colors[1]], linecolor='white', linewidth=2 )

***feature Open, high, low are highly correlated to Target feature Close. we can use either one of the feature for prediction to avoid multicollinearity***

# Train Test Split

In [ ]:
X=df1[['volume','open']]
y=df1['close']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, shuffle=False, random_state=42)

# Normalizing the values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test =scaler.transform(X_test)

# Model Creation  
## basic Linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import set_config
model = LinearRegression()
model.fit(X_train,y_train)
set_config(display='diagram')
pred=model.predict(X_test)
sc=np.round(model.score(X_test, y_test),2) * 100
r2=np.round(r2_score(y_test,pred),2)
mse=np.round(mean_squared_error(y_test,pred),2)
mae=np.round(mean_squared_error(y_test,pred),2)

In [ ]:
fig=plt.figure(figsize=(15,8))
p=pd.Series(pred, index=y_test.index)
plt.plot(y_test)
plt.plot(p)
plt.legend(['y_test','predicted'])
plt.title("Compare test and predicted values", size=20, weight='bold')
plt.text(x=800000, y=600,s='Accuracy score : {} %'.format(sc))
plt.text(x=800000, y=580,s='R2 Score : {}'.format(r2))
plt.text(x=800000, y=560,s='Mean Squared error : {}'.format(mse))
plt.text(x=800000, y=540,s='Mean Absolute error : {}'.format(mae))

# Try with LSTM

In [ ]:
X=df1[['open','high']]
y=df1['close']
length=100
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=False, random_state=42)
training_set = X.iloc[:1000].values
test_set = X.iloc[1000:].values
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
test_set_scaled=sc.transform(test_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(length, len(training_set)):
    X_train.append(training_set_scaled[i-length:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = []
y_test = []
for i in range(length, len(test_set)):
    X_test.append(test_set_scaled[i-length:i, 0])
    y_test.append(test_set_scaled[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs = 100, batch_size = 32)

In [ ]:
loss = pd.DataFrame(model.history.history)
fig=plt.figure(figsize=(15,8))
plt.title("Validation loss Vs Trainiinverse_transformss", size=20, weight='bold')
plt.plot(loss)

In [ ]:
pred=model.predict(X_test)
test=pd.DataFrame(columns=['test','pred'])
test['test']=y_test
test['pred']=pred.flatten()
test

In [ ]:
fig=plt.figure(figsize=(15,8))
plt.title("Test vs Predicted value", size=20, weight='bold')
plt.plot(test)
plt.legend(['test','predict'])
r2=np.round(r2_score(y_test,pred),2)
mse=np.round(mean_squared_error(y_test,pred),2)
mae=np.round(mean_squared_error(y_test,pred),2)
plt.text(x=500, y=1.5,s='R2 Score : {}'.format(r2))
plt.text(x=500, y=1.35,s='Mean Squared error : {}'.format(mse))
plt.text(x=500, y=1.25,s='Mean Absolute error : {}'.format(mae))

# Try with FBProphet

In [ ]:
from fbprophet import Prophet
df_p = df1[['date','close']]
df_p.columns=['ds','y']

split_data = df_p.index.max()-100000
train = df_p.loc[df_p.index<=split_data].copy()
test=df_p.loc[df_p.index>split_data].copy()
train.set_index('ds',inplace=True)
test.set_index('ds',inplace=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

#Model creation
model=Prophet()
model.fit(train)

#model prediction
pred=model.predict(test)

In [ ]:
test_pred=pd.DataFrame(columns=['ds','test','predict','pred_lower','pred_high'], index=test.index)
test_pred['test']=test['y']
test_pred['ds']=test['ds']
test_pred['predict']=pred['yhat']
test_pred['pred_lower']=pred['yhat_lower']
test_pred['pred_high']=pred['yhat_upper']

In [ ]:
#plotting Test vs Predicted
fig=plt.figure(figsize=(15,8))
plt.title("Test Vs Prediction", size=20, weight='bold')
sns.lineplot(data=test_pred,x='ds',y='test')
sns.lineplot(data=test_pred,x='ds',y='predict')

r2=np.round(r2_score(test_pred['test'],test_pred['predict']),2)
mse=np.round(mean_squared_error(test_pred['test'],test_pred['predict']),2)
mae=np.round(mean_squared_error(test_pred['test'],test_pred['predict']),2)
plt.text(x=mdates.datestr2num('2016-10'), y=700,s='R2 Score : {}'.format(r2))
plt.text(x=mdates.datestr2num('2016-10'), y=680,s='Mean Squared error : {}'.format(mse))
plt.text(x=mdates.datestr2num('2016-10'), y=660,s='Mean Absolute error : {}'.format(mae))

In [ ]:
fig=plt.figure(figsize=(15,8))
plt.title("Test Vs Prediction", size=20, weight='bold')
sns.lineplot(data=train,x='ds',y='y')
sns.lineplot(data=test_pred,x='ds',y='predict')
sns.lineplot(data=test_pred,x='ds',y='test', alpha=0.5, ls='--', color='black')

r2=np.round(r2_score(test_pred['test'],test_pred['predict']),2)
mse=np.round(mean_squared_error(test_pred['test'],test_pred['predict']),2)
mae=np.round(mean_squared_error(test_pred['test'],test_pred['predict']),2)
plt.text(x=mdates.datestr2num('2015'), y=300,s='R2 Score : {}'.format(r2))
plt.text(x=mdates.datestr2num('2015'), y=260,s='Mean Squared error : {}'.format(mse))
plt.text(x=mdates.datestr2num('2015'), y=220,s='Mean Absolute error : {}'.format(mae))
plt.legend(['Train','Predict','Test'])